# Import libraries

In [57]:
import urllib, re
from urllib.request import urljoin
from urllib.request import urlopen
from bs4 import BeautifulSoup as Soup
import requests
import pandas as pd
from tqdm.auto import tqdm

In [53]:
def crawl(pages, depth=None):
    indexed_url = [] # a list for the main and sub-HTML websites in the main website
    for i in range(depth):
        for page in pages:
            if page not in indexed_url:
                indexed_url.append(page)
                try:
                    c = urllib.request.urlopen(page)
                except:
                    print ("Could not open %s" % page)
                    continue
                soup = BeautifulSoup(c.read(), "lxml")
                links = soup('a') #finding all the sub_links
                for link in links:
                    if 'href' in dict(link.attrs):
                        url = urljoin(page, link['href'])
                        if url.find("'") != -1:
                                continue
                        url = url.split('#')[0] 
                        if url[0:4] == 'http':
                                indexed_url.append(url)
        pages = indexed_url
    return indexed_url

In [63]:
pagelist=["http://yusufsn.staff.ums.ac.id/"]
urls = crawl(pagelist, depth=2)

In [64]:
urls

['http://yusufsn.staff.ums.ac.id/',
 'http://yusufsn.staff.ums.ac.id/',
 'http://yusufsn.staff.ums.ac.id/assignments',
 'http://yusufsn.staff.ums.ac.id/extra-credit',
 'http://yusufsn.staff.ums.ac.id/contact-me',
 'http://yusufsn.staff.ums.ac.id/home/IMG_20140414_110048.jpg?attredirects=0',
 'http://yusufsn.staff.ums.ac.id/contact-me',
 'http://yusufsn.staff.ums.ac.id/classroom-news',
 'http://yusufsn.staff.ums.ac.id/my-calendar',
 'http://yusufsn.staff.ums.ac.id/my-reading-list',
 'http://yusufsn.staff.ums.ac.id/useful-links',
 'http://yusufsn.staff.ums.ac.id/classroom-pictures',
 'http://yusufsn.staff.ums.ac.id/my-files',
 'http://yusufsn.staff.ums.ac.id/poll',
 'http://www.ums.ac.id',
 'http://fki.ums.ac.id',
 'https://yusufums.wordpress.com/',
 'http://informatika.ums.ac.id',
 'http://yusufsn.staff.ums.ac.id/student-of-the-month/studentoftheoddsemester20162017',
 'http://yusufsn.staff.ums.ac.id/student-of-the-month',
 'http://yusufsn.staff.ums.ac.id/word-of-the-week/analisis',
 'ht

In [65]:
len(urls)

38

In [62]:
data = []
url = 'https://github.com/advisories?page='
for i in tqdm(range(1,41)):
    r = requests.get(url + str(i))
    soup = Soup(r.content,"html.parser" )
    
    for div in soup.find_all("div", class_= "lh-condensed flex-auto"):
        for link in div.find_all("a"):
            title = link.get_text().replace("\t","").replace("\n","")
            urls = "https://github.com/" + link['href']
        for ttt in div.find_all("div", class_="mt-1 text-small text-gray"):
            txtdata = (ttt.get_text().replace("\n",""))
            try:
                cve = re.findall("([A-Z]+\-.*?)\s+",txtdata)[0]
            except:
                cve = "-"
            try:
                severity = re.findall("\((\w.+severity)\)",txtdata)[0]
            except:
                severity = "-"
            try:
                pubdate = re.findall("published\s+(\w.+[0-9]{4})",txtdata)[0]
            except:
                pubdate = "-"
            try:
                package = re.findall("\((\w+)\)",txtdata)[0]
            except:
                package = "-"
    
        data.append([title, urls, cve, severity, pubdate, package])

df = pd.DataFrame(data, columns=['title','url','cve_id','severity','published_date','package'])
print ("Extraction finish")


Extraction finish


In [63]:
df.head(20)

,title,url,cve_id,severity,published_date,package
0,XSS via JQLite DOM manipulat...,https://github.com//advisories/GHSA-5cp4-xmrw-...,GHSA-5cp4-xmrw-59wf,Moderate severity,"Aug 5, 2020",npm
1,CSRF Vulnerability ...,https://github.com//advisories/GHSA-whrh-9j4q-...,GHSA-whrh-9j4q-g7ph,Moderate severity,"Aug 5, 2020",npm
2,Reset Password / Login vulne...,https://github.com//advisories/GHSA-wfm4-pq59-...,CVE-2020-15132,Moderate severity,"Aug 5, 2020",Composer
3,CSRF on PgHero gem ...,https://github.com//advisories/GHSA-v6fx-752r-...,CVE-2020-16253,Moderate severity,"Aug 5, 2020",RubyGems
4,CSRF in Field Test ...,https://github.com//advisories/GHSA-w542-cpp9-...,CVE-2020-16252,Moderate severity,"Aug 5, 2020",RubyGems
5,[CVE-2020-15109] Ability to ...,https://github.com//advisories/GHSA-3mvg-rrrw-...,CVE-2020-15109,Moderate severity,"Aug 4, 2020",RubyGems
6,CSRF vulnerability ...,https://github.com//advisories/GHSA-wwrj-35w6-...,CVE-2020-15135,Moderate severity,"Aug 4, 2020",npm
7,Denial of service in fastify...,https://github.com//advisories/GHSA-xw5p-hw6r-...,CVE-2020-8192,Moderate severity,"Aug 5, 2020",npm
8,Prototype Pollution in expre...,https://github.com//advisories/GHSA-9wcg-jrwf-...,CVE-2020-7699,High severity,"Aug 5, 2020",npm
9,Code execution in Spring Int...,https://github.com//advisories/GHSA-86qr-9vqc-...,CVE-2020-5413,Low severity,"Aug 5, 2020",Maven
